In [1]:
from IPython.display import Image

- parallel(wide) think
    - ToT: tree of thought
    - deepmind, AlphaGeometry
        - llm 负责提出一些合理的 construction，然后结合原始的题目的形式化描述，送给符号引擎
        - How Deep Think works: extending Gemini’s parallel “thinking time”
    - manus wide research: https://manus.im/zh-cn/blog/introducing-wide-research
- deep think
    - reasoning model

### AlphaGeometry

- 语言模型 (LLM - Gemini-based): 扮演“直觉”和“创造者”的角色。它的任务不是进行严格的逻辑推理，而是根据现有信息，提出最有可能通向解题路径的辅助构造（Auxiliary Constructions）。它擅长模式识别和启发式搜索。
- 符号引擎 (DDAR - Deductive Database Arithmetic Reasoning): 扮演“逻辑学家”和“验证者”的角色。它接收一个包含所有已知点和几何关系的集合（事实库），然后通过固定的、高效的演绎规则（比如三点共线、四点共圆的判定，全等相似的传递等），计算出这个事实库能推导出的所有结论（这个过程叫“演绎闭包” - Deduction Closure）。它的过程是完备且可靠的，但没有创造性。
- Shared Knowledge Ensemble of Search Trees (SKEST),
- 第一步
    - 问题形式化 (Formalization): 首先，自然语言描述的几何问题被翻译成 AG2 的领域特定语言（DSL）。
        - Given a triangle ABC with two equal sides AB = AC, prove that angles B and C are equal
        - triangle a b c; a b = a c ? eqangle b a b c c b c a
    - 首次 DDAR 运行: 在不添加任何辅助点的情况下，符号引擎 DDAR 会对初始问题进行一次完整的演绎闭包计算。
    - 生成“分析字符串” (Analysis String): 这是 LLM 和 DDAR 交互的关键接口，也是 AG2 相比 AG1 的重大改进。DDAR 会生成一个包含三部分信息的字符串，反馈给 LLM：
        - S1: 仅从初始条件出发，DDAR 能证明的所有事实。
        - S2: 假设结论也成立，DDAR 能反向推导出的所有事实。
        - S3: 通过检查数值化图表（diagram）发现为真的事实（几何位置关系）。
        - 这个分析字符串的格式如论文所述：`<problem_statement> serialized(S1) serialized(S2 - S1) serialized(S3 - S2)`。它告诉 LLM：“这是原题，这是我们已经确切知道的，这是如果我们假设结论成立能得到的东西，这是图上看起来像真的东西。现在，请你告诉我下一步该怎么走？”
- 第 1 步：LLM 提出候选构造
    - 构建 Prompt: 系统将“分析字符串”作为上下文，向 LLM 发出请求。这个 Prompt 的核心是：“请基于以上信息，提出一个或多个有用的辅助构造。”
    - LLM 生成候选: LLM 会生成一系列候选的辅助构造，同样使用 AG2 的 DSL。例如，它可能会生成：
        - x00 D : on_line A B, perp C D A B (作 C 关于 AB 的垂线，垂足为 D)
        - x01 E : circumcenter A B C (作 △ABC 的外心 E)
        - ...等等

In [3]:
Image(url='https://lh3.googleusercontent.com/CXoZ8QVYA7wKFPt3RurU7Z0SDyp32YQS9gJaEwE-U1AtjAQ-eXEaGxnOSTUH01oyN7YOxz-BILe390w2wHVEFF7XPmCOzqr0QMBroKc4J5kPFyqYVqU=w1232-rw', 
      width=700)